### Steps 

- Find a dataset of 10,00 most popular packages (use python or js as there the languages I know) 
- Train a BPE tokenizer on source code dataset
- Train an LLM on the tokenized dataset (they use GPT2-774M which has 36 layers, 20 attention heads and a max sequence length of 1024, learning rate of 0.0001, weight decay of 0.01 and cosine learning rate decay schedule) - They trained it on four RTX800 GPUS for one month. I wont be able to do that, so I will used a smaller dataset and a smaller model which requires less compute.
 

The authors in the paper take the original program, apply a security test (codeQL) where the program fails due a bug, then they apply a prompt for the llm to fix the bug from the report codeQL gives. The llm outputs a patch which is then merged into the original program. The program is then tested again to see if the bug is fixed.

Ouput - Fixed programm, patch and the test results.

Challenges the authors seemed to face:
- Prompt engineering. Lots of models can only take in a certain amount of tokens for the context they can ingest. 
- If the program is too big, the model wont be able to process it, meaning the program needs to be split into smaller chunks.
- Prompts should follow LLM documentation - https://www.promptingguide.ai/applications/coding


Evaluation:
- The authors generate buggy code examples and then tested codex(openAi) LLM which is used by Github Copilot. They then compared the results of the LLM to the results of the authors GPT2.



### Generate dataset



In [1]:
%pip install pandas

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 325.1 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.8 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 294.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB 667.8 kB/s eta 0:00:18
   ---------------------------------------- 0.1/11.5 MB 660.6 kB/s eta 0:00:18
   ---------------------------------------- 0.1/11.5 MB 590.8 kB/s eta 0:00:20
   ---------------------------------------- 0.1/11.5 MB 590.8 kB/s eta 0:00:20
   ---------------------------------------- 0.1/11.5 MB 532.5 kB/s eta 0:00:22
    ----


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\summe\OneDrive\Documents\New-Projects\patch_zero\env\Scripts\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

In [2]:
test = pd.read_pickle('../data/bugfixes_test.pickle')
train = pd.read_pickle('../data/bugfixes_train.pickle')
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
Index: 14118 entries, 1790 to 625636
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   after_merge                      14118 non-null  object
 1   before_merge                     14118 non-null  object
 2   filename                         14118 non-null  object
 3   full_file_code_after_merge       14118 non-null  object
 4   full_file_code_before_merge      14118 non-null  object
 5   function_name                    14118 non-null  object
 6   url                              14118 non-null  object
 7   source code and errors           14118 non-null  object
 8   full_traceback                   14118 non-null  object
 9   traceback_type                   14118 non-null  object
 10  before_merge_without_docstrings  14118 non-null  object
 11  after_merge_without_docstrings   14118 non-null  object
 12  before_merge_docstrings          

,after_merge,before_merge,filename,full_file_code_after_merge,full_file_code_before_merge,function_name,url,source code and errors,full_traceback,traceback_type,before_merge_without_docstrings,after_merge_without_docstrings,before_merge_docstrings,after_merge_docstrings,path_to_snippet_before_merge,path_to_snippet_after_merge
1790,"def plot(result_pickle_file_path, show, plot_s...","def plot(result_dict_file, show, plot_save_fil...",rqalpha/mod/rqalpha_mod_sys_analyser/__init__.py,# -*- coding: utf-8 -*-\n#\n# Copyright 2017 R...,# -*- coding: utf-8 -*-\n#\n# Copyright 2017 R...,plot,https://github.com/ricequant/rqalpha/issues/109,"[{'piece_type': 'other', 'piece_content': 'rqa...","Traceback (most recent call last):\nFile ""c:\p...",TypeError,"def plot(result_dict_file, show, plot_save_fil...","def plot(result_pickle_file_path, show, plot_s...",[[sys_analyser] draw result DataFrame],[[sys_analyser] draw result DataFrame],buggy_snippets_files/e93817735d3042d739fe86677...,buggy_snippets_files/e93817735d3042d739fe86677...
2433,"def stream_logs(self):\n """"""Stream ...","def stream_logs(self):\n """"""Stream ...",binderhub/build.py,"""""""\nContains build of a docker image from a g...","""""""\nContains build of a docker image from a g...",Build.stream_logs,https://github.com/jupyterhub/binderhub/issues...,"[{'piece_type': 'error message', 'piece_conten...",/ # jupyter-repo2docker https://github.com/yuv...,FileNotFoundError,def stream_logs(self):\n \n ...,def stream_logs(self):\n \n ...,[],[],buggy_snippets_files/8241189c4267b81254c9ed07a...,buggy_snippets_files/8241189c4267b81254c9ed07a...
26618,"def addRecentProjectFile(self, projectFile...","def addRecentProjectFile(self, projectFile...",meshroom/ui/app.py,import logging\nimport os\nimport argparse\n\n...,import logging\nimport os\nimport argparse\n\n...,MeshroomApp.addRecentProjectFile,https://github.com/alicevision/meshroom/issues...,"[{'piece_type': 'error message', 'piece_conten...","[2020-05-23 16:12:48,660][ERROR] Traceback (mo...",OSError,"def addRecentProjectFile(self, projectFile...","def addRecentProjectFile(self, projectFile...",[],[],buggy_snippets_files/faddf4c059bd32cc1cad1a1ea...,buggy_snippets_files/faddf4c059bd32cc1cad1a1ea...
26622,"def addSfmAugmentation(self, withMVS=False...","def addSfmAugmentation(self, withMVS=False...",meshroom/ui/reconstruction.py,import logging\nimport os\nfrom threading impo...,import logging\nimport os\nfrom threading impo...,Reconstruction.addSfmAugmentation,https://github.com/alicevision/meshroom/issues...,"[{'piece_type': 'error message', 'piece_conten...","Traceback (most recent call last):\nFile ""C:\U...",RuntimeError,"def addSfmAugmentation(self, withMVS=False...","def addSfmAugmentation(self, withMVS=False...",[],[],buggy_snippets_files/dffb9602005cbea45f7d0c6d2...,buggy_snippets_files/dffb9602005cbea45f7d0c6d2...
28217,def load_pymathics_doc(self):\n if ...,def load_pymathics_doc(self):\n if ...,mathics/doc/doc.py,#!/usr/bin/env python3\n# -*- coding: utf-8 -*...,#!/usr/bin/env python3\n# -*- coding: utf-8 -*...,MathicsMainDocumentation.load_pymathics_doc,https://github.com/mathics/Mathics/issues/906,"[{'piece_type': 'error message', 'piece_conten...",$ mathicsserver\nwarning: database file /home/...,KeyError,def load_pymathics_doc(self):\n if ...,def load_pymathics_doc(self):\n if ...,[],[],buggy_snippets_files/c98cf1a03e1d7e716b228fbe8...,buggy_snippets_files/c98cf1a03e1d7e716b228fbe8...


In [56]:
def get_pre_merge_buggy_code(df: pd.DataFrame, n: int) -> pd.DataFrame:
    return list(df.iloc[:, 1])[:n]

train_data = get_pre_merge_buggy_code(train, 3)
train_data

['def plot(result_dict_file, show, plot_save_file):\n    """\n    [sys_analyser] draw result DataFrame\n    """\n    import pandas as pd\n    from .plot import plot_result\n\n    result_dict = pd.read_pickle(result_dict_file)\n    plot_result(result_dict, show, plot_save_file)',
 '    def stream_logs(self):\n        """Stream a pod\'s log."""\n        for line in self.api.read_namespaced_pod_log(\n                self.name,\n                self.namespace,\n                follow=True,\n                _preload_content=False):\n\n            self.progress(\'log\', line.decode(\'utf-8\'))',
 '    def addRecentProjectFile(self, projectFile):\n        projectFile = QUrl(projectFile).path()\n        projects = self._recentProjectFiles()\n\n        # remove duplicates while preserving order\n        from collections import OrderedDict\n        uniqueProjects = OrderedDict.fromkeys(projects)\n        projects = list(uniqueProjects)\n        # remove previous usage of the value\n        if pr

In [66]:
import re
import pprint
from collections import defaultdict

def get_stats(train_data:list):
    """
    Given a vocabulary (dictionary mapping words to frequency counts), returns a 
    dictionary of tuples representing the frequency count of pairs of characters 
    in the vocabulary.
    """
    pairs = defaultdict(int)
    for word, freq in train_data.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_corp_dict(pair, v_in):
    """
    Given a pair of characters and a corpus dict, returns a new corpus dict with the 
    pair of characters merged together wherever they appear.
    """
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

def get_corpus_dict(data: list):
    """
    Given a list of functions, return a dictionary of words mapping to their frequency count.
    """
    corpus_dict = defaultdict(int)
    for func in data:
        for word in func.split(): # Remove whitespace
            corpus_dict[' '.join(list(word)) + ' </w>'] += 1
    return corpus_dict

def byte_pair_encoding(train_data: list, n:int):
    """
    Given a list of strings and an integer n, returns a list of n merged pairs
    of characters found in the vocabulary of the input data.
    """
    corpus_dict = get_corpus_dict(train_data)    
    for _ in range(n):
        pairs = get_stats(corpus_dict)
        best = max(pairs, key=pairs.get)
        corpus_dict = merge_corp_dict(best, corpus_dict)
    return corpus_dict

bpe_pairs = byte_pair_encoding(train_data, len(train_data))
bpe_pairs

{'d e f </w>': 3,
 'p l o t ( re s u l t _ d i ct _ f i l e , </w>': 1,
 's h o w , </w>': 2,
 'p l o t _ s a v e _ f i l e ) : </w>': 1,
 '" " " </w>': 2,
 '[ s y s _ a n a l y s e r ] </w>': 1,
 'd r a w </w>': 1,
 're s u l t </w>': 1,
 'D a t a F r a m e </w>': 1,
 'i m p o r t </w>': 3,
 'p a n d a s </w>': 1,
 'a s </w>': 1,
 'p d </w>': 1,
 'f ro m </w>': 2,
 '. p l o t </w>': 1,
 'p l o t _ re s u l t </w>': 1,
 're s u l t _ d i ct </w>': 1,
 '= </w>': 8,
 'p d . re a d _ p i c k l e ( re s u l t _ d i ct _ f i l e ) </w>': 1,
 'p l o t _ re s u l t ( re s u l t _ d i ct , </w>': 1,
 'p l o t _ s a v e _ f i l e ) </w>': 1,
 's t re a m _ l o g s ( s e l f ) : </w>': 1,
 '" " " S t re a m </w>': 1,
 'a </w>': 1,
 "p o d ' s </w>": 1,
 'l o g . " " " </w>': 1,
 'f o r </w>': 2,
 'l i n e </w>': 1,
 'i n </w>': 4,
 's e l f . a p i . re a d _ n a m e s p a c e d _ p o d _ l o g ( </w>': 1,
 's e l f . n a m e , </w>': 1,
 's e l f . n a m e s p a c e , </w>': 1,
 'f o l l o w = 